# Astro Data Archive example: Retrieve metadata for images of a list of coordinates
Knut Olsen

## Imports and setup

In [1]:
import pandas as pd
import numpy as np
import astropy.units as u
import requests

**fieldlist** contains the columns of metadata that we want to retrieve.

**searchlist** contains the search constraints that we want to apply at any given coordinate.  Here we specify that we want images from either DECam or Mosaic-2, we only want image data products (*image*) that have been processed (*instcal*), we only want science images (*object*), we want exposure time of at least 30 seconds, and we want a range of dates.

In [2]:
natroot = 'https://astroarchive.noirlab.edu'
fieldlist = ["archive_filename","ra_center","dec_center","proposal","ifilter","MJD-OBS","OBJECT","survey",
             "dateobs_center","instrument","proc_type","exposure","obs_type","prod_type"]
searchlist = [["instrument", "decam", "mosaic_2"], 
              ["prod_type","image"],
              ["proc_type","instcal"],[
                  "obs_type","object"],
              ["exposure",30.,36000.],[
                  "caldat","2000-01-01","2023-10-01"]]

adsurl = f'{natroot}/api/adv_search'

## Assemble coordinates of Sculptor Group galaxies
We'll store a list of coordinates in a Pandas DataFrame.

In [3]:
data = {
    'Name': ['NGC 45', 'NGC 55', 'NGC 247', 'NGC 253', 'NGC 300', 'NGC 7793'],
    'ra': [3.5127, 3.7258, 11.7864, 11.888, 13.7256, 359.4562],
    'dec': [-23.1806, -39.1965, -20.7604, -25.2882, -37.6842, -32.5909]
}

df = pd.DataFrame(data)
print(df)

       Name        ra      dec
0    NGC 45    3.5127 -23.1806
1    NGC 55    3.7258 -39.1965
2   NGC 247   11.7864 -20.7604
3   NGC 253   11.8880 -25.2882
4   NGC 300   13.7256 -37.6842
5  NGC 7793  359.4562 -32.5909


## Loop over the coordinates and conduct a box search
For each coordinate in our DataFrame, we add a search constraint to only find images with image centers within a 0.5 degree-wide box of our coordinate.  The search is made by passing the list of columns that we want and our search constraints to the API. 

In [4]:
boxwidth = 0.5 # degrees
apiurl = f'{adsurl}/find/?limit=10000000'
output = []
for i in df.index:
    ra0 = df.iloc[i]['ra']
    dec0 = df.iloc[i]['dec']
    ramin = ra0 - (boxwidth/2./np.cos(dec0*u.deg)).value
    ramax = ra0 + (boxwidth/2./np.cos(dec0*u.deg)).value
    decmin = dec0 - boxwidth/2.
    decmax = dec0 + boxwidth/2.

    srch = searchlist.copy()
    srch.append(["ra_center",ramin,ramax])
    srch.append(["dec_center",decmin,decmax])

    jj = {"outfields" : fieldlist, "search" : srch}
    try:
        res = pd.DataFrame(requests.post(apiurl,json=jj).json()[1:])
        output.append(res)
    except:
        print('Problem encountered with '+df.iloc[i]['Name'])

## Assemble output
We'll concatenate the list of output DataFrames into a single DataFrame *dfout*.

In [5]:
dfout = pd.concat(output)

In [6]:
dfout

,ra_center,ifilter,dec_center,proposal,archive_filename,survey,exposure,obs_type,prod_type,proc_type,instrument,dateobs_center,OBJECT,MJD-OBS
0,3.442125,i SDSS c6019,-23.117139,2006A-0086,/net/archive/pipeline/Q20110624/C4M06A/2006062...,None,300.0,object,image,instcal,mosaic_2,2006-06-26T10:31:36.100000Z,"NGC 45 - DelRA = 0.0, DelDec = 0.0",53912.436877
1,3.455250,g SDSS c6017,-23.134806,2006A-0086,/net/archive/pipeline/Q20110623/C4M06A/2006062...,None,300.0,object,image,instcal,mosaic_2,2006-06-26T09:56:52.200000Z,"NGC 45 - DelRA = 43.2, DelDec = -64.8",53912.412757
2,3.429125,g SDSS c6017,-23.098389,2006A-0086,/net/archive/pipeline/Q20110623/C4M06A/2006062...,None,300.0,object,image,instcal,mosaic_2,2006-06-26T10:03:34.300000Z,"NGC 45 - DelRA = -43.2, DelDec = 64.8",53912.417411
3,3.428958,i SDSS c6019,-23.098861,2006A-0086,/net/archive/pipeline/Q20110624/C4M06A/2006062...,None,300.0,object,image,instcal,mosaic_2,2006-06-26T10:45:03.300000Z,"NGC 45 - DelRA = -43.2, DelDec = 64.8",53912.446219
4,3.455292,r SDSS c6018,-23.134861,2006A-0086,/net/archive/pipeline/Q20110625/C4M06A/2006062...,None,300.0,object,image,instcal,mosaic_2,2006-06-26T10:17:31.700000Z,"NGC 45 - DelRA = 43.2, DelDec = -64.8",53912.427103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,359.484542,V Harris c6026,-32.545444,2004B-0075,/net/archive/pipeline/Q20110831/C4M04B/2005011...,None,300.0,object,image,instcal,mosaic_2,2005-01-18T01:40:08.300000Z,"NGC 7793 - DelRA = 80.0, DelDec = 120.0",53388.067804
54,359.487042,I c6028,-32.578306,2004B-0227,/net/archive/pipeline/Q20110908/C4M04B/2004120...,None,300.0,object,image,instcal,mosaic_2,2004-12-12T02:13:35.100000Z,"NGC 7793 - DelRA = 21.6, DelDec = 32.4",53351.091031
55,359.488333,V Harris c6026,-32.663000,2004B-0075,/net/archive/pipeline/Q20110831/C4M04B/2005011...,None,300.0,object,image,instcal,mosaic_2,2005-01-12T01:30:41.300000Z,"NGC 7793 - DelRA = 160.0, DelDec = -240.0",53382.061242
56,359.494042,I c6028,-32.605306,2004B-0227,/net/archive/pipeline/Q20110908/C4M04B/2004120...,None,300.0,object,image,instcal,mosaic_2,2004-12-12T02:00:11.200000Z,"NGC 7793 - DelRA = 43.2, DelDec = -64.8",53351.081726
